https://www.kaggle.com/datasets/brianblakely/top-100-songs-and-lyrics-from-1959-to-2019

Genre classification
https://www.kaggle.com/datasets/mateibejan/multilingual-lyrics-for-genre-classification


In [ ]:
!unzip lyrics.zip

Archive:  lyrics.zip
replace all_songs_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: all_songs_data.csv      
replace all_songs_data.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: all_songs_data.json     
replace all_songs_data_processed.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: all_songs_data_processed.csv  


In [ ]:
import pandas as pd

song_processed = pd.read_csv('all_songs_data_processed.csv')

In [ ]:
song_processed

,Album,Album URL,Artist,Featured Artists,Lyrics,Media,Rank,Release Date,Song Title,Song URL,Writers,Year,Verbs,Nouns,Adverbs,Corpus,Word Counts,Unique Word Counts
0,Battle of New Orleans,https://genius.com/albums/Johnny-horton/Battle...,Johnny Horton,[],In 1814 we took a little trip Along with Colo...,[{'native_uri': 'spotify:track:0dwpdcQkeZqpuoA...,1,1959-04-01,The Battle Of New Orleans,https://genius.com/Johnny-horton-the-battle-of...,"[{'api_path': '/artists/561913', 'header_image...",1959.0,take take take catch fire keep be be fire begi...,trip bacon bean town gun comin while river of'...,along as ago once more so as ago once more the...,take little trip Colonel Jackson mighty Missis...,435,155
1,That’s All,https://genius.com/albums/Bobby-darin/That-s-all,Bobby Darin,[],"Oh the shark, babe Has such teeth, dear And he...",[{'native_uri': 'spotify:track:3E5ndyOfO6vFDEI...,2,NaN,Mack The Knife,https://genius.com/Bobby-darin-mack-the-knife-...,"[{'api_path': '/artists/218851', 'header_image...",1959.0,have show have keep know bite billow spread be...,shark babe tooth jackknife babe sight shark to...,pearly just so never never now just just down ...,oh shark babe tooth dear show pearly white jac...,224,145
2,“Mr Personality’s” 15 Big Hits,https://genius.com/albums/Lloyd-price/Mr-perso...,Lloyd Price,[],Over and over I tried to prove my love to you ...,"[{'provider': 'youtube', 'start': 0, 'type': '...",3,NaN,Personality,https://genius.com/Lloyd-price-personality-lyrics,"[{'api_path': '/artists/355804', 'header_image...",1959.0,try prove do say get get do cause get get do s...,love friend fool fool personality personality ...,over over over over over over so over now over...,try prove love friend say fool fool cause get ...,215,55
3,The Greatest Hits Of Frankie Avalon,https://genius.com/albums/Frankie-avalon/The-g...,Frankie Avalon,[],"Hey, Venus! Oh, Venus! Venus, if you will Ple...",[],4,NaN,Venus,https://genius.com/Frankie-avalon-venus-lyrics,"[{'api_path': '/artists/1113175', 'header_imag...",1959.0,send thrill want make take place ask promise g...,girl girl kiss arm girl charm girl sunlight ha...,surely too always as long surely too always as...,hey Venus oh Venus Venus send little girl thri...,166,74
4,Paul Anka Sings His Big 15,https://genius.com/albums/Paul-anka/Paul-anka-...,Paul Anka,[],I'm just a lonely boy Lonely and blue I'm all ...,[],5,NaN,Lonely Boy,https://genius.com/Paul-anka-lonely-boy-lyrics,[],1959.0,do get think want love love kiss hold like hea...,boy nothing moment love night day boy nothing ...,just all just all just so hard just all,lonely boy lonely blue get think want love yes...,140,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6287,One Thing At A Time,https://genius.com/albums/Morgan-wallen/One-th...,Morgan Wallen,[],When it comes to hitchin' the boat up Backin' ...,"[{'attribution': 'morgan wallen', 'provider': ...",95,2023-01-31,I Wrote the Book,https://genius.com/Morgan-wallen-i-wrote-the-b...,"[{'api_path': '/artists/675171', 'header_image...",2023.0,come find catch write write wanna learn throw ...,boat ramp truck bunch log bunch hog book book ...,damn enough never too damn enough probably dam...,come hitchin boat Backin ramp old truck find b...,374,134
6288,BZRP Music Sessions,https://genius.com/albums/Bizarrap/Bzrp-music-...,Bizarrap and Shakira,[],"(Pa' tipos como tú, uh-uh-uh-uh) Oh-oh (Oh-oh...","[{'provider': 'youtube', 'start': 0, 'type': '...",96,2023-01-11,"Bzrp Music Sessions, Vol. 53",https://genius.com/Bizarrap-and-shakira-shakir...,"[{'api_path': '/artists/1405', 'header_image_u...",2023.0,eso que que que que heriste mujere like person...,tipos tipos vuelvo quiero que baby como a ti p...,also,Pa tipos como tú uh uh uh uh oh oh oh oh Pa ti...,405,192
6289,L3*,https://genius.com/albums/Latto/L3,Latto featuring Cardi B,"[{'api_path': '/artists/621678', 'header_image...",(Go Grizz) Ah (Squat made the beat) What's hap...,"[{'provider': 'youtube', 'start

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

data = []
for year in range(2020, 2025):
    url = f"https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_{year}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    rows = soup.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        if len(cols) >= 3:
            rank = cols[0].text.strip()
            title = cols[1].text.strip()
            artist = cols[2].text.strip()

            data.append([rank, title, artist, year])
    time.sleep(0.2)

df = pd.DataFrame(data, columns=['Rank', 'Title', 'Artist', 'Year'])
df['Rank'] = pd.to_numeric(df['Rank'], errors='raise').astype('Int64')
df['Year'] = pd.to_numeric(df['Year'], errors='raise').astype('Int64')

In [ ]:
df = df.sort_values(by=['Year', 'Rank'])
df = df.reset_index(drop=True)

In [ ]:
'''
Clean and standardize song titles and artist names. Remove unwanted characters, accents, and additional
information like featured artists or parenthetical text.
'''

import pandas as pd
import re
import unicodedata

def sanitize_string(s):
    if not isinstance(s, str):
        return ""
    s = ''.join(c for c in unicodedata.normalize('NFKD', s) if not unicodedata.combining(c) and c.isascii())
    s = s.replace(" ", "-")
    s = re.sub(r'[^a-zA-Z0-9-"\'-]', '', s)  # Keeps " and '
    return s

def clean_artist(artist):
    if not isinstance(artist, str):
        return ""
    artist = re.split(r'\b(and|featuring)\b', artist, 1)[0].strip()
    artist = re.sub(r'\(.*?\)', '', artist).strip()
    return sanitize_string(artist)

def clean_title(title):
    if not isinstance(title, str):
        return ""
    title = re.sub(r'\(.*?\)', '', title).strip()
    return sanitize_string(title)

df['Cleaned_Title'] = df['Title'].apply(clean_title)
df['Cleaned_Artist'] = df['Artist'].apply(clean_artist)


In [ ]:
df_2020 = df[df["Year"] == 2020]
df_2021 = df[df["Year"] == 2021]
df_2022 = df[df["Year"] == 2022]
df_2023 = df[df["Year"] == 2023]
df_2024 = df[df["Year"] == 2024]

In [ ]:
# lyrics scrapping:

API_HOST = "genius-song-lyrics1.p.rapidapi.com"
API_KEY = ""

dataframes = [df_2020, df_2021, df_2022, df_2023, df_2024]

# Clean all DataFrames
for df in dataframes:
    df["Cleaned_Title"] = df["Title"].apply(clean_title)
    df["Cleaned_Artist"] = df["Artist"].apply(clean_artist)
    df["link"] = ""
    df["lyrics"] = ""


def fetch_lyrics(df):
    for index, row in df.iterrows():
        song_name = row['Cleaned_Title']
        artist_name = row['Cleaned_Artist']

        conn = http.client.HTTPSConnection(API_HOST)

        headers = {
            'x-rapidapi-key': API_KEY,
            'x-rapidapi-host': API_HOST
        }


        query = quote(f"{song_name} {artist_name}")
        conn.request("GET", f"/search/?q={query}", headers=headers)
        res = conn.getresponse()
        data = res.read()
        search_results = json.loads(data.decode("utf-8"))

        hits = search_results.get("hits", [])

        if hits:
            song_id = hits[0].get("result", {}).get("id")
            song_url = hits[0].get("result", {}).get("url")
        else:
            song_id, song_url = None, None

        if song_id:
            print(f"Found Song: {song_url}")
            df.at[index, 'link'] = song_url

            conn.request("GET", f"/song/lyrics/?id={song_id}", headers=headers)
            res = conn.getresponse()
            data = res.read()
            lyrics_response = json.loads(data.decode("utf-8"))
            lyrics = lyrics_response.get("lyrics")

            if lyrics:
                df.at[index, 'lyrics'] = lyrics
                print(f"Lyrics found for {song_name}")
            else:
                df.at[index, 'lyrics'] = None
                print("Lyrics not found.")
        else:
            df.at[index, 'link'] = None
            df.at[index, 'lyrics'] = None
            print(f"No match found for {song_name} by {artist_name}")


        time.sleep(0.5)


In [ ]:
df_2021.to_csv('df_2021.csv', index=False)
df_2022.to_csv('df_2022.csv', index=False)
df_2023.to_csv('df_2023.csv', index=False)
df_2024.to_csv('df_2024.csv', index=False)

In [ ]:
dataframes = [df_2020, df_2021, df_2022, df_2023, df_2024]
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df.to_csv("combined_songs_2020_2024.csv")

In [ ]:
combined_df = combined_df.sort_values(by=['Year', 'Rank'])
combined_df = combined_df.reset_index(drop=True)

In [ ]:
combined_df.loc[0, 'lyrics']

'{\'_type\': \'lyrics\', \'api_path\': \'/songs/5049949/lyrics\', \'lyrics\': {\'body\': {\'html\': \'<p>[Intro]<br>\\nYeah<br>\\n<br>\\n[Verse 1]<br>\\n<a href="/18550610/The-weeknd-blinding-lights/Ive-been-tryna-call" data-id="18550610" class="has_pending_edits" data-editorial-state="accepted" data-classification="accepted">I\\\'ve been tryna call</a><br>\\n<a href="/18547744/The-weeknd-blinding-lights/Ive-been-on-my-own-for-long-enough" data-id="18547744" data-editorial-state="accepted" data-classification="accepted">I\\\'ve been on my own for long enough</a><br>\\nMaybe you can show me\\u2005how\\u2005to love, maybe<br>\\n<a href="/18601361/The-weeknd-blinding-lights/Im-goin-through-withdrawals" data-id="18601361" class="has_comments" data-editorial-state="accepted" data-classification="accepted">I\\\'m\\u2005goin\\\' through withdrawals</a><br>\\n<a href="/29486210/The-weeknd-blinding-lights/You-dont-even-have-to-do-too-much" data-id="29486210" data-editorial-state="pending" data-

In [ ]:
import string
def clean_lyrics(text):
    match = re.search(r"'html': (.*?), 'path':", text, re.DOTALL)
    if match:
        extracted_data = match.group(1)
        extracted_data = re.sub(r"<.*?>", "", extracted_data)
        extracted_data = re.sub(r"\\u\d{4}", " ", extracted_data)
        extracted_data = re.sub(r"\[.*?\]", "", extracted_data)
        extracted_data = extracted_data.replace("\\'", "'")
        extracted_data = re.sub(r"\n|\\n", " ", extracted_data)
        extracted_data = re.sub(r"\\u\d{3,4}", "", extracted_data)
        extracted_data = extracted_data.replace("\\", "")
        extracted_data = re.sub(r"\s+", " ", extracted_data).strip()
        extracted_data = extracted_data.translate(str.maketrans('', '', string.punctuation))
        return extracted_data
    return text

combined_df["lyrics_cleaned"] = combined_df["lyrics"].apply(clean_lyrics)

In [ ]:
combined_df.loc[50, 'lyrics_cleaned']

' SpeakerBangerz Look uh look Heart been broke so many times I dont know what to believe Mama say it’s my fault its my fault I wear my heart on my sleeve Think its best I put my heart on ice heart on ice ’cause I cant breathe Ima put my heart on ice heart on ice its gettin the best of me While in the cell with Lil Hakeem after I slapped him I had told him I dont know how you get down with them clowns but Im a soldier No one could understand I had way too much aggression That built over the years from my abandoned adolescence See I done been lied to backstabbed and heartbroken I wanted to cry but I was too afraid to open Prayin’ one day I’d find a peace of mind by the ocean I spent all my time committing crimes to get closer While at my nana house I play the couch starin at the ceiling Tryin’ not to get in my feelings Thinkin of a way I could make these millions Maybe thatll take this pain away and clear up all these rainy days yeah Heart been broke so many times I dont know what to bel

In [ ]:
lyrics = pd.read_csv("final_lyrics_cleaned_2020-2024.csv")

In [ ]:
lyrics

,Unnamed: 0.1,Unnamed: 0,Rank,Title,Artist,Year,Cleaned_Title,Cleaned_Artist,link,lyrics_cleaned
0,0,0,1,"""Blinding Lights""",The Weeknd,2020,"""Blinding-Lights""",The-Weeknd,https://genius.com/The-weeknd-blinding-lights-...,Yeah Ive been tryna call Ive been on my own f...
1,1,1,2,"""Circles""",Post Malone,2020,"""Circles""",Post-Malone,https://genius.com/Post-malone-circles-lyrics,Oh oh oh Oha oh oh Oha oh oh oh oh We couldnt...
2,2,2,3,"""The Box""",Roddy Ricch,2020,"""The-Box""",Roddy-Ricch,https://genius.com/Roddy-ricch-the-box-lyrics,Pullin out the coupe at the lot Told em fuck ...
3,3,3,4,"""Don't Start Now""",Dua Lipa,2020,"""Don't-Start-Now""",Dua-Lipa,https://genius.com/Dua-lipa-dont-start-now-lyrics,If you dont wanna see me Did a full oneeighty...
4,4,4,5,"""Rockstar""",DaBaby featuring Roddy Ricch,2020,"""Rockstar""",DaBaby,https://genius.com/Dababy-rockstar-lyrics,Woo woo I pull up like How you pull up Baby H...
...,...,...,...,...,...,...,...,...,...,...
495,495,495,96,"""Bulletproof""",Nate Smith,2024,"""Bulletproof""",Nate-Smith,https://genius.com/Nate-smith-bulletproof-lyrics,Usually these ol neon lights Are pretty good ...
496,496,496,97,"""Fe!n""",Travis Scott featuring Playboi Carti,2024,"""Fen""",Travis-Scott,https://genius.com/Travis-scott-fe-n-lyrics,Just come outside for the night Yeah Take you...
497,497,497,98,"""The Painter""",Cody Johnson,2024,"""The-Painter""",Cody-Johnson,https://genius.com/Cody-johnson-the-painter-ly...,She talks about the future like shes flippin ...
498,498,498,99,"""Down Bad""",Taylor Swift,2024,"""Down-Bad""",Taylor-Swift,https://genius.com/Taylor-swift-down-bad-lyrics,Did you really beam me up In a cloud of spark...


In [ ]:
updated_songs_df = pd.concat([song_processed, lyrics], ignore_index=True)
lyrics['Lyrics'] = lyrics['Lyrics'].astype(str).apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))
updated_songs_df.to_csv("all_songs_cleaned_1959-2024.csv")

In [ ]:
import pandas as pd

lyrics = pd.read_csv("songs_genre_labeled_2020-2024.csv")

In [ ]:
lyrics

,Year,Title,Artist,Rank,lyrics_cleaned,cluster,predicted_genre,predicted_genre_sl,predicted_genre_svc,predicted_genre_nb,predicted_genre_xgb
0,2020,"""Blinding Lights""",The Weeknd,1,Yeah Ive been tryna call Ive been on my own f...,4,R&B,Pop,Pop,Pop,Pop
1,2020,"""Circles""",Post Malone,2,Oh oh oh Oha oh oh Oha oh oh oh oh We couldnt...,4,R&B,Pop,Pop,Rock,Pop
2,2020,"""The Box""",Roddy Ricch,3,Pullin out the coupe at the lot Told em fuck ...,4,Hip-Hop,Pop,Pop,Pop,Pop
3,2020,"""Don't Start Now""",Dua Lipa,4,If you dont wanna see me Did a full oneeighty...,4,Hip-Hop,Pop,Pop,Pop,Pop
4,2020,"""Rockstar""",DaBaby featuring Roddy Ricch,5,Woo woo I pull up like How you pull up Baby H...,4,Hip-Hop,Pop,Pop,Pop,Pop
...,...,...,...,...,...,...,...,...,...,...,...
495,2024,"""Bulletproof""",Nate Smith,96,Usually these ol neon lights Are pretty good ...,4,Hip-Hop,Pop,Pop,Pop,Rock
496,2024,"""Fe!n""",Travis Scott featuring Playboi Carti,97,Just come outside for the night Yeah Take you...,4,Hip-Hop,Pop,Pop,Pop,Pop
497,2024,"""The Painter""",Cody Johnson,98,She talks about the future like shes flippin ...,4,Hip-Hop,Rock,Rock,Rock,Rock
498,2024,"""Down Bad""",Taylor Swift,99,Did you really beam me up In a cloud of spark...,4,Hip-Hop,Metal,Metal,Pop,Rock
